<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myStocks_Portfolio_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
import os

# Install es_MX
!/usr/share/locales/install-language-pack en_IN
!dpkg-reconfigure locales

# Restart Python process to pick up the new locales
os.kill(os.getpid(), 9)
'''

'\nimport os\n\n# Install es_MX\n!/usr/share/locales/install-language-pack en_IN\n!dpkg-reconfigure locales\n\n# Restart Python process to pick up the new locales\nos.kill(os.getpid(), 9)\n'

In [2]:
!pip install pyxirr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.2/541.2 kB 9.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Mounted at /content/drive


In [4]:
# Import the function from the script
import myStocks_Portfolio_Analysis_v2

# Call the function to get the DataFrame
df_portfolio_features = myStocks_Portfolio_Analysis_v2.get_portfolio_features_v2()

Run date time (IST): 2024-07-24 22:56:25
-------------------
qualified stocks: 80
with latest results: 78
still star stocks: 46
-------------------
Initial Investment:  ₹88,92,324.00
CY Investment:  ₹1,02,03,324.00
Reserve:  ₹3,07,032.00
Current:  ₹1,12,08,270.00
-------------------
Today PnL: ₹89,395.00 (0.8%)
Current PnL: ₹5,65,946.00 (5.55%)
CY Booked + Current PnL: ₹10,04,946.00 (9.85%)
-------------------
Total profit:  ₹9,65,414.00
Total loss:  -₹3,99,467.00
-------------------
Total Booked + Current PnL: ₹23,15,946.00 (26.04%)
Total Booked PnL: ₹17,50,000.00 (19.68%)
Curr Year Booked PnL: ₹4,39,000.00 (3.92%)
Prev Year Booked PnL: ₹13,11,000.00 (14.74%)
Est NTT:  ₹1,53,62,513.00
Est NTT PnL: ₹41,54,243.00 (37.06%)
-------------------
Est LTT:  ₹2,05,70,745.00
Est LTT PnL: ₹93,62,475.00 (83.53%)
Deployed:  ₹89,02,000.00
Current:  ₹1,12,08,270.00
CAGR/XIRR %: 27.07%


In [5]:
# all portfolio
df_portfolio_features = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
data_table.DataTable(df_portfolio_features, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
19,CAMS,4071.40,2424.02,4450.00,35.62,29.55,UP,25,101785.0,41184.0,...,0.93,4.35,AVR,BTT,63.0,84.17,1,1,M,MISC
35,ICICIPRULI,694.25,508.03,747.00,23.75,49.33,UP,198,137462.0,36872.0,...,1.26,3.53,AV40,BTT,46.0,44.95,1,0,H,INSURANCE
22,DABUR,634.15,523.35,710.00,15.93,10.91,UP,175,110976.0,19390.0,...,1.02,1.46,AV40,BTT,22.0,28.61,1,1,H,FMCG
49,MARICO,657.95,530.08,725.00,20.06,14.79,UP,200,131590.0,25574.0,...,1.21,1.91,AV40,BTT,29.0,34.26,1,1,H,FMCG
63,SHALBY,278.55,264.74,327.00,-0.52,NaN,UP,302,84122.0,4171.0,...,0.77,0.29,AVSP,NTT,4.0,51.07,1,1,H,HEALTHCARE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,AAVAS,1759.25,1840.86,3195.80,12.05,-38.34,UP,89,156573.0,-7264.0,...,1.44,-0.06,AVSP,ATH,-6.0,33.72,1,1,M,FINANCE
31,HDFCBANK,1604.05,1569.25,2170.00,5.39,-11.34,NaN,272,436302.0,9466.0,...,4.00,0.06,AV40,BTT,9.0,17.48,1,1,H,BANKS
10,BANDHANBNK,188.10,227.40,400.00,-10.43,-27.89,NaN,827,155559.0,-32501.0,...,1.43,-0.19,AVSP,NTT,-25.0,8.54,1,0,H,BANKS
69,TANLA,949.00,969.42,2007.31,-1.45,-20.18,DOWN,166,157534.0,-3390.0,...,1.45,-0.02,AAR,ATH,-3.0,18.67,1,1,L,IT


In [6]:
# AAR analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AAR') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
65,SONACOMS,687.45,578.05,800.00,11.24,5.20,UP,175,120304.0,19145.0,...,1.10,0.97,AAR,NTT,21.0,35.72,1,1,M,AUTO
52,NEOGEN,1604.00,1489.90,1901.29,7.65,71.90,NaN,68,109072.0,7759.0,...,1.00,0.38,AAR,ATH,8.0,34.32,1,0,L,CHEMICALS
43,JUBLFOOD,582.25,487.35,703.00,14.17,6.33,UP,224,130424.0,21258.0,...,1.20,0.79,AAR,NTT,23.0,35.37,1,0,L,RESTAURANTS
23,DEN,54.36,52.79,75.00,0.67,2.54,UP,1600,86976.0,2512.0,...,0.80,0.08,AAR,NTT,2.0,41.93,1,0,L,ENTERTAINMENT
28,HAPPSTMNDS,827.85,919.80,1496.99,-1.33,-12.25,NaN,105,86924.0,-9655.0,...,0.80,-0.14,AAR,ATH,-8.0,11.62,1,1,M,IT
69,TANLA,949.00,969.42,2007.31,-1.45,-20.18,DOWN,166,157534.0,-3390.0,...,1.45,-0.02,AAR,ATH,-3.0,18.67,1,1,L,IT


In [7]:
# AVSP analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AVSP') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
63,SHALBY,278.55,264.74,327.00,-0.52,NaN,UP,302,84122.0,4171.0,...,0.77,0.29,AVSP,NTT,4.0,51.07,1,1,H,HEALTHCARE
16,BIOCON,348.00,291.09,397.00,24.73,-16.72,UP,516,179568.0,29366.0,...,1.65,1.16,AVSP,NTT,32.0,59.34,1,1,H,PHARMA
59,ROSSARI,820.20,770.50,965.00,7.92,-29.49,UP,190,155838.0,9443.0,...,1.43,0.34,AVSP,NTT,9.0,22.30,1,1,M,CHEMICALS
25,GICRE,376.80,335.06,441.00,14.53,1.54,UP,437,164662.0,18241.0,...,1.51,0.65,AVSP,NTT,19.0,97.50,1,0,M,INSURANCE
76,UTIAMC,1016.60,946.90,1207.00,16.99,-16.34,UP,158,160623.0,11013.0,...,1.47,0.37,AVSP,NTT,11.0,40.52,1,1,M,FINANCE
17,BOROLTD,374.20,342.46,462.00,6.31,NaN,NaN,347,129847.0,11013.0,...,1.19,0.36,AVSP,NTT,11.0,24.61,1,1,M,GLASS
66,STARHEALTH,613.20,596.56,761.00,10.09,-13.92,NaN,268,164338.0,4460.0,...,1.51,0.11,AVSP,NTT,4.0,22.82,1,1,H,INSURANCE
40,IRCTC,974.20,904.87,1269.00,8.12,0.49,UP,156,151975.0,10815.0,...,1.39,0.24,AVSP,BTT,11.0,58.34,1,1,H,TRAVEL
15,BERGEPAINT,529.95,558.54,680.00,-3.29,-30.11,DOWN,317,167994.0,-9063.0,...,1.54,-0.19,AVSP,NTT,-8.0,16.29,1,1,H,PAINTS
62,SFL,1016.50,1079.73,1287.00,-3.10,19.50,NaN,187,190086.0,-11824.0,...,1.74,-0.23,AVSP,NTT,-10.0,16.54,1,0,M,MISC
